# HandsOn Week 11
Selamat datang di HandsOn Week 11, yaitu tentang pemrosesan data terdistribusi menggunakan Spark. Untuk tujuan pembelajaran, seperti biasa, kita akan menggunakan *pseudo-distributed mode* (single node cluster) di VM yang telah disediakan. Dengan kode yang *similar* di cluster komputer dengan *n* workers, maka komputasi akan tersebar ke *n* workers tersebut. Adapun yang akan kita coba kali ini adalah melakukan komputasi menggunakan RDD dan DataFrame. Berikut catatan-catatan yang perlu kamu perhatikan dalam hands-on ini:
1. Untuk menjalankan Apache Spark dalam bahasa python di VM, ketikkan perintah ```pyspark``` di terminal.
2. Dari semua Milestone, data input yang digunakan adalah data "purchases.txt" yang telah diletakkan di HDFS. Oleh karena itu, pastikan hadoop service kamu berjalan (```start-dfs.sh```, ```start-yarn.sh```, ```jps```). Untuk membaca data dari HDFS, lihat kembali di slide perkuliahan.
3. Untuk Milestone 1, 2 dan 3, kalian perlu untuk mencatat waktu yang diperlukan saat melakukan MapReduce menggunakan hadoop streaming jar di hands-on sebelumnya. Waktu bisa dihitung dari selisih "waktu awal" dan "waktu akhir" yang tampak di terminal saat kalian selesain melakukan MapReduce -atau menggunakan cara lain yang masih *acceptable*-. (lihat ilustrasi di bawah).
4. Lakukan zip file jupyter notebook ini beserta gambar-gambar yang diperlukan -screenshot waktu proses MapReduce Hadoop jar-, dan submit ke portal kuliah stei dengan format nama "**HandsOnWeek11_NIM_NamaLengkap.zip**". Pastikan file jupyter notebook yang kamu zip dalam kondisi memiliki output per cellnya (tidak kosong karena belum dijalankan). <br>

<img title="Waktu Awal" align="left" src="waktu_awal.JPG" alt="Drawing" style="width: 600px;"/>
<img title="Waktu Akhir" align="left" src="waktu_akhir.JPG" alt="Drawing" style="width: 600px;"/>

## Milestone 1
Kerjakan Milestone 1 pada HandsOn Week10, akan tetapi menggunakan RDD Spark. Catat waktu (bandingkan) yang dibutuhkan (dalam detik) antara: "MapReduce menggunakan hadoop streaming jar" dengan yang akan kamu proses menggunakan RDD Spark ini.

In [29]:
import time

start_time = time.time()

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')
rdd11 = purchases.map(lambda x: (x.split('\t')[3], float(x.split('\t')[4])))

def func1(x):
    if 'Toys' in x[0] or 'Consumer Electronics' in x[0]:
        return x
    
rdd12 = rdd11.filter(func1)
rdd13 = rdd12.reduceByKey(lambda a, b: a + b)

print("Hasil:", rdd13.collect())
print("Waktu yang diperlukan dengan Hadoop:", "219 seconds")
print("Waktu yang diperlukan dengan RDD Spark:", (time.time() - start_time), " seconds")

time_m1 = time.time() - start_time

Hasil: [('Consumer Electronics', 57452374.12999931), ('Toys', 57463477.10999949)]
Waktu yang diperlukan dengan Hadoop: 219 seconds
Waktu yang diperlukan dengan RDD Spark: 83.99946737289429  seconds


#### Screenshot Waktu Proses MapReduce Hadoop jar

<img title="Waktu Awal M1" align="left" src="waktu_awal_m1.png" alt="Drawing" style="width: 600px;"/>
<img title="Waktu Akhir M1" align="left" src="waktu_akhir_m1.png" alt="Drawing" style="width: 600px;"/>

## Milestone 2
Kerjakan Milestone 2 pada HandsOn Week10, akan tetapi menggunakan RDD Spark. Catat waktu (bandingkan) yang dibutuhkan (dalam detik) antara: "MapReduce menggunakan hadoop streaming jar" dengan yang akan kamu proses menggunakan RDD Spark ini.

In [30]:
start_time = time.time()

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')
rdd21 = purchases.map(lambda x: ((x.split('\t')[2], x.split('\t')[3]), float(x.split('\t')[4])))
def func2(x):
    if 'Atlanta' in x[0] or 'Miami' in x[0] or 'San Francisco' in x[0]:
        return x
    
rdd22 = rdd21.filter(func2)
rdd23 = rdd22.reduceByKey(lambda a,b: a+b)
rdd24 = rdd23.map(lambda x: (x[0][0], (x[1], x[0][1])))
rdd25 = rdd24.reduceByKey(max)
rdd26 = rdd25.map(lambda x: (x[0], x[1][0], x[1][1]))

print("Hasil:", rdd26.collect())
print("Waktu yang diperlukan dengan Hadoop:", "161 seconds")
print("Waktu yang diperlukan dengan RDD Spark:", (time.time() - start_time), " seconds")

time_m2 = time.time() - start_time

Hasil: [('Atlanta', 578389.7300000001, 'Garden'), ('San Francisco', 571250.8600000006, 'Health and Beauty'), ('Miami', 572410.7800000007, 'Cameras')]
Waktu yang diperlukan dengan Hadoop: 161 seconds
Waktu yang diperlukan dengan RDD Spark: 98.45878887176514  seconds


#### Screenshot Waktu Proses MapReduce Hadoop jar

<img title="Waktu Awal M2" align="left" src="waktu_awal_m2.png" alt="Drawing" style="width: 600px;"/>
<img title="Waktu Akhir M2" align="left" src="waktu_akhir_m2.png" alt="Drawing" style="width: 600px;"/>

## Milestone 3
Kerjakan Milestone 3 pada HandsOn Week10, akan tetapi menggunakan RDD Spark. Catat waktu (bandingkan) yang dibutuhkan (dalam detik) antara: "MapReduce menggunakan hadoop streaming jar" dengan yang akan kamu proses menggunakan RDD Spark ini.

In [31]:
start_time = time.time()

init_time_1 = time.strptime('09:01', '%H:%M')
final_time_1 = time.strptime('10:00', '%H:%M')
init_time_2 = time.strptime('10:01', '%H:%M')
final_time_2 = time.strptime('11:00', '%H:%M')

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')

def func3(x):
    now_time = time.strptime(x[1], '%H:%M')
    if init_time_1 <= now_time and now_time <= final_time_1:
        return ('09:01-10:00', 1)
    elif init_time_2 <= now_time and now_time <= final_time_2:
        return ('10:01-11:00', 1)
    else:
        return ('NONE', 1)
    
rdd31 = purchases.map(lambda x: x.split('\t'))
rdd32 = rdd31.map(func3)
rdd33 = rdd32.filter(lambda x: x[0] != 'NONE')
rdd34 = rdd33.reduceByKey(lambda a, b: a + b)

print("Hasil:", rdd34.collect())
print("Waktu yang diperlukan dengan Hadoop:", "901 seconds")
print("Waktu yang diperlukan dengan RDD Spark:", (time.time() - start_time), " seconds")

time_m3 = time.time() - start_time

Hasil: [('09:01-10:00', 459775), ('10:01-11:00', 459825)]
Waktu yang diperlukan dengan Hadoop: 901 seconds
Waktu yang diperlukan dengan RDD Spark: 310.66835713386536  seconds


#### Screenshot Waktu Proses MapReduce Hadoop jar

<img title="Waktu Awal M3" align="left" src="waktu_awal_m3.png" alt="Drawing" style="width: 600px;"/>
<img title="Waktu Akhir M3" align="left" src="waktu_akhir_m3.png" alt="Drawing" style="width: 600px;"/>

## Milestone 4
Milestone ini dibagi menjadi 4.1, 4.2 dan 4.3 yang masing-masing secara berturut-turut adalah mengerjakan ulang Milestone 1, 2 dan 3 di atas (menggunakan RDD Spark), akan tetapi menggunakan trik "**persist() RDD**" untuk mempercepat prosesnya. Kamu bisa melakukan "**persist**" untuk RDD mana saja yang kamu anggap dapat memberikan waktu proses tercepat.

In [33]:
# Tuliskan code kamu di sini

# Milestone 1
start_time = time.time()

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')
rdd11 = purchases.map(lambda x: (x.split('\t')[3], float(x.split('\t')[4])))

def func1(x):
    if 'Toys' in x[0] or 'Consumer Electronics' in x[0]:
        return x
    
rdd12 = rdd11.filter(func1)
rdd13 = rdd12.reduceByKey(lambda a, b: a + b).persist()

print("Hasil:", rdd13.collect())
print("Waktu Milestone 1:", time_m1, " vs. Waktu Milestone 4.1:", (time.time() - start_time), " seconds")

# Milestone 2
start_time = time.time()

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')
rdd21 = purchases.map(lambda x: ((x.split('\t')[2], x.split('\t')[3]), float(x.split('\t')[4])))
def func2(x):
    if 'Atlanta' in x[0] or 'Miami' in x[0] or 'San Francisco' in x[0]:
        return x
    
rdd22 = rdd21.filter(func2)
rdd23 = rdd22.reduceByKey(lambda a,b: a+b).persist()
rdd24 = rdd23.map(lambda x: (x[0][0], (x[1], x[0][1])))
rdd25 = rdd24.reduceByKey(max)
rdd26 = rdd25.map(lambda x: (x[0], x[1][0], x[1][1]))

print("Hasil:", rdd26.collect())
print("Waktu Milestone 2:", time_m2, " vs. Waktu Milestone 4.2:", (time.time() - start_time), " seconds")

# Milestone 3
start_time = time.time()

init_time_1 = time.strptime('09:01', '%H:%M')
final_time_1 = time.strptime('10:00', '%H:%M')
init_time_2 = time.strptime('10:01', '%H:%M')
final_time_2 = time.strptime('11:00', '%H:%M')

purchases = sc.textFile('hdfs://localhost:9000/purchases/purchases.txt')

def func3(x):
    now_time = time.strptime(x[1], '%H:%M')
    if init_time_1 <= now_time and now_time <= final_time_1:
        return ('09:01-10:00', 1)
    elif init_time_2 <= now_time and now_time <= final_time_2:
        return ('10:01-11:00', 1)
    else:
        return ('NONE', 1)
    
rdd31 = purchases.map(lambda x: x.split('\t'))
rdd32 = rdd31.map(func3)
rdd33 = rdd32.filter(lambda x: x[0] != 'NONE').persist()
rdd34 = rdd33.reduceByKey(lambda a, b: a + b)

print("Hasil:", rdd34.collect())
print("Waktu Milestone 3:", time_m3, " vs. Waktu Milestone 4.3:", (time.time() - start_time), " seconds")

Hasil: [('Consumer Electronics', 57452374.12999931), ('Toys', 57463477.10999949)]
Waktu Milestone 1: 83.99974822998047  vs. Waktu Milestone 4.1: 82.76808834075928  seconds
Hasil: [('Atlanta', 578389.7300000001, 'Garden'), ('San Francisco', 571250.8600000006, 'Health and Beauty'), ('Miami', 572410.7800000007, 'Cameras')]
Waktu Milestone 2: 98.45949745178223  vs. Waktu Milestone 4.2: 96.9364185333252  seconds
Hasil: [('09:01-10:00', 459775), ('10:01-11:00', 459825)]
Waktu Milestone 3: 310.6691563129425  vs. Waktu Milestone 4.3: 310.2760920524597  seconds


## Milestone 5
Milestone ini dibagi menjadi 5.1, 5.2 dan 5.3 yang masing-masing secara berturut-turut adalah mengerjakan ulang Milestone 1, 2 dan 3 di atas, akan tetapi menggunakan DataFrame dari Apache Spark. Catat waktu yang diperlukan untuk masing-masing proses (5.1, 5.2 dan 5.3).

In [19]:
spark = SparkSession.builder \
    .appName("W11 Milestone 5") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

purchases = spark.read.csv('hdfs://localhost:9000/purchases/purchases.txt', inferSchema=True, sep='\t')
purchases.createOrReplaceTempView('purchases')
purchases.show(5)

+-------------------+-----+----------+-------------------+------+----------+
|                _c0|  _c1|       _c2|                _c3|   _c4|       _c5|
+-------------------+-----+----------+-------------------+------+----------+
|2012-01-01 00:00:00|09:00|  San Jose|     Men's Clothing|214.05|      Amex|
|2012-01-01 00:00:00|09:00|Fort Worth|   Women's Clothing|153.57|      Visa|
|2012-01-01 00:00:00|09:00| San Diego|              Music| 66.08|      Cash|
|2012-01-01 00:00:00|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|
|2012-01-01 00:00:00|09:00|     Omaha|Children's Clothing|235.63|MasterCard|
+-------------------+-----+----------+-------------------+------+----------+
only showing top 5 rows



### Milestone 5.1

In [20]:
start_time = time.time()
spark.sql("select _c3 as product, sum(_c4) as value from purchases where _c3 like '%Toys%' or _c3 like '%Consumer Electronics%' group by _c3").show(5)
print("Waktu Milestone 5.1:", (time.time() - start_time), " seconds")

+--------------------+-------------------+
|             product|              value|
+--------------------+-------------------+
|Consumer Electronics|5.745237412999931E7|
|                Toys|5.746347710999949E7|
+--------------------+-------------------+

Waktu Milestone 5.1: 43.17247176170349  seconds


### Milestone 5.2

In [21]:
start_time = time.time()
spark.sql("select temp1._c2 as city, temp1.sum_value as value, temp1._c3 as product from (select _c2, sum(_c4) as sum_value, _c3 from purchases where _c2=='Miami' or _c2=='Atlanta' or _c2=='San Francisco' group by _c2, _c3) as temp1 join (select _c2, max(sum_value) as sum_value from (select _c2, sum(_c4) as sum_value, _c3 from purchases where _c2=='Miami' or _c2=='Atlanta' or _c2=='San Francisco' group by _c2, _c3) group by _c2) as temp2 on temp1._c2==temp2._c2 and temp1.sum_value==temp2.sum_value").show(5)
print("Waktu Milestone 5.2:", (time.time() - start_time), " seconds")

+-------------+-----------------+-----------------+
|         city|            value|          product|
+-------------+-----------------+-----------------+
|San Francisco|571250.8600000006|Health and Beauty|
|        Miami|572410.7800000007|          Cameras|
|      Atlanta|578389.7300000001|           Garden|
+-------------+-----------------+-----------------+

Waktu Milestone 5.2: 69.4471788406372  seconds


### Milestone 5.3

In [22]:
start_time = time.time()
spark.sql("select time, sum(cnt) as count from (select case when cast(substr(_c1, 1, 2) as int)==9 and (cast(substr(_c1, 4, 2) as int) between 1 and 59) then '09:01-10:00' when cast(substr(_c1, 1, 2) as int)==10 and (cast(substr(_c1, 4, 2) as int)==0) then '09:01-10:00' when cast(substr(_c1, 1, 2) as int)==10 and (cast(substr(_c1, 4, 2) as int) between 1 and 59) then '10:01-11:00' when cast(substr(_c1, 1, 2) as int)==11 and (cast(substr(_c1, 4, 2) as int)==0) then '10:01-11:00' end as time, 1 as cnt from purchases) where time is not Null group by time").show(5)
print("Waktu Milestone 5.3:", (time.time() - start_time), " seconds")

+-----------+------+
|       time| count|
+-----------+------+
|10:01-11:00|459825|
|09:01-10:00|459775|
+-----------+------+

Waktu Milestone 5.3: 44.2259566783905  seconds
